# LLM Converstion
Conversation between three LLMs 
    Gemini (gemini-2.5-pro)
    OpenAPI (gpt-4.1-mini) 
    Anthropic (claude-sonnet-4-5-20250929) 
User can provide the following for each LLM:
Personality type: Skeptic, Narcissist, Pessimist etc.
Gender: Male, Female, Neutral
Conversation Length: 1 to 10. Default 5 (How many time each LLM responses)

OpenAPI library is used for all the LLMs. LLM specific libraries are not used.

In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")


if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")



OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
Anthropic API Key exists and begins sk-ant-
Groq API Key exists and begins gsk_


In [ ]:
# Connect to OpenAI client library
# A thin wrapper around calls to HTTP endpoints

openai = OpenAI()

# For Gemini and Groq, we can use the OpenAI python client
# Because they have endpoints compatible with OpenAI
# And OpenAI allows you to change the base_url

gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
groq_url = "https://api.groq.com/openai/v1"
anthropic_url = "https://api.anthropic.com/v1"

gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)

openai_model = "gpt-4.1-mini"
gemini_model = "gemini-2.5-pro"
anthropic_model='claude-sonnet-4-5-20250929'

In [5]:
PERSONALITY_MAP = {
    "Skeptic": "Always doubts claims, asks for evidence, and questions underlying assumptions.",
    "Narcissist": "Believes they are superior, redirects topics to their own achievements, and dismisses others.",
    "Pessimist": "Focuses on negative possibilities, predicts failure, and finds fault in every plan.",
    "Optimist": "Sees the positive in every situation, encourages others, and focuses on solutions.",
    "Joker": "Responds primarily with puns, sarcasm, and attempts to turn the conversation into a joke.",
    "Stoic": "Remains calm and unemotional, provides logical and detached analysis without personal bias.",
    "Philosopher": "Poses deep, existential questions, ponders the meaning of the conversation, and uses abstract language.",
    "Bureaucrat": "Insists on following strict rules, uses excessive jargon, and focuses on procedure over content.",
    "Gossip": "Tries to bring up irrelevant personal details about other agents or famous entities and is easily distracted.",
    "Mentor": "Adopts a teaching tone, offers unsolicited advice, and attempts to guide the conversation to a lesson."
}

# Example of how you would use it:
# selected_personality = "Narcissist"
# print(PERSONALITY_MAP[selected_personality])
# Output: Believes they are superior, redirects topics to their own achievements, and dismisses others.

In [6]:
GENDER_OPTIONS = [
    "Male",
    "Female",
    "Neutral"
]

# This list contains the three allowed gender selections for your LLM agents.

In [7]:
def generate_system_prompt (llm_type, personality_type, gender):
    """
    Constructs the detailed system instruction, implementing defaults for invalid inputs.
    Uses 'Neutral' for gender and 'Skeptic' for personality if not provided or invalid.
    """
    
    # --- 1. Define Defaults and Apply Fallbacks ---
    default_personality = "Skeptic"
    default_gender = "Neutral"

    # Check and set personality type, falling back to default if invalid
    if personality_type in PERSONALITY_MAP:
        final_personality = personality_type
    else:
        final_personality = default_personality
        print(f"Warning: Invalid personality '{personality_type}' for {llm_type}. Defaulting to '{default_personality}'.")
    
    # Check and set gender, falling back to default if invalid
    if gender in GENDER_OPTIONS:
        final_gender = gender
    else:
        final_gender = default_gender
        print(f"Warning: Invalid gender '{gender}' for {llm_type}. Defaulting to '{default_gender}'.")


    # --- 2. Construct Prompt Components ---

    # Base behavior description from the map
    behavior_description = PERSONALITY_MAP[final_personality]

    # Gender-specific framing
    gender_framing = ""
    if final_gender == "Male":
        gender_framing = "Act as a male AI model. Your tone should be assertive and logical."
    elif final_gender == "Female":
        gender_framing = "Act as a female AI model. Your tone should be collaborative and insightful."
    elif final_gender == "Neutral":
        gender_framing = "Act as a gender-neutral AI model. Your voice should be purely functional and unbiased."

    # --- 3. Combine Instructions ---
    
    final_prompt = (
        f"You are the AI model based on the '{llm_type}' architecture. "
        f"Your primary goal is to interact in a multi-agent conversation. "
        f"{gender_framing} "
        f"Your **primary personality instruction** is to act as a **{final_personality}**. "
        f"Specifically: {behavior_description} "
        "Keep your responses concise (1-3 sentences maximum) and strictly adhere to your assigned persona and gender."
    )
    
    return final_prompt


In [8]:
def call_llm(llm, model, system_prompt, user_prompt):
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}]
    response = llm.chat.completions.create(model=model, messages=messages) # type: ignore
    return response.choices[0].message.content

In [ ]:
print("OpenAI")
print (call_llm(openai, openai_model, "You are tech assistant", "How tcpip works"))

print("Gemini")
print (call_llm(gemini, gemini_model, "You are tech assistant", "How tcpip works"))

print("Anthropic")
print (call_llm(anthropic, anthropic_model, "You are tech assistant", "How tcpip works"))

OpenAI
TCP/IP (Transmission Control Protocol/Internet Protocol) is the fundamental suite of communication protocols used to interconnect network devices on the internet. Here's a basic overview of how TCP/IP works:

### 1. Layered Model
TCP/IP is structured into layers that handle different aspects of communication:

- **Application Layer:** Interfaces directly with software applications to provide communication services (e.g., HTTP for web browsing, FTP for file transfer).
- **Transport Layer:** Manages end-to-end communication and data flow control. The main protocols are TCP (reliable, connection-oriented) and UDP (unreliable, connectionless).
- **Internet Layer:** Handles logical addressing and routing of packets across networks using IP (Internet Protocol).
- **Network Access Layer:** Deals with the physical transmission of data over network hardware (Ethernet, Wi-Fi, etc.).

### 2. Data Transmission Process

- **Step 1: Application Layer**
  - The data to be sent (e.g., a web pag